In [ ]:
from tqdm.notebook import trange # Progress bar
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import geometry
import matplotlib.pyplot as plt
import matplotlib as mpl

### Define Color Bar

In [ ]:
# LinearSegmentedColormap method

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
ax = plt.subplot()
clist=['blue','limegreen','red']
# N is the number of color
newcmap = LinearSegmentedColormap.from_list('mycolmap',clist,N=512)
im = ax.imshow(np.arange(100).reshape((10, 10)),cmap=newcmap)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.show()


In [ ]:
import cartopy.crs as ccrs # Module for drawing maps
import cartopy.feature as cfeatur
from mpl_toolkits.axes_grid1 import make_axes_locatable


data = pd.read_excel(r'C:\\Users\\*****\\data_class_analyes.xlsx', sheet_name=0)

config = {
    "font.family":'Times New Roman',
    "mathtext.fontset": 'stix',
    "font.serif": ['SimSun'],
}
mpl.rcParams.update(config)
mpl.rcParams['axes.unicode_minus']=False
# Function to adding Points to a Map
def point_plot(poin_lon, poin_lat, col_value, poin_title:str):
    fig, ax = plt.subplots(figsize=(15, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.set_xlim([-180, 180])
    ax.set_ylim([-90, 90])
    plt.title(poin_title)
    mapbar = ax.scatter(poin_lon, poin_lat, c=col_value, s=9, cmap=newcmap, vmin=0, vmax=100)
    ax.add_feature(cfeatur.OCEAN.with_scale('10m'), zorder=2)  
    ax.add_feature(cfeatur.LAKES.with_scale('10m'), zorder=2) 
    ax.gridlines(draw_labels=True, linestyle='--', lw = 0.3)  
    cax = fig.add_axes([ax.get_position().x0, ax.get_position().y0-0.05, ax.get_position().width, 0.02]) # set color bar
    clb = plt.colorbar(mapbar, cax=cax, orientation = 'horizontal')
    clb.set_label('Herbivory')
    plt.savefig(r'C:\\Users\\****\\photo\\' + poin_title + '.png', dpi=600, bbox_inches = 'tight')
    
point_plot(poin_lon = data['Longitude'], poin_lat = data['Latitude'], col_value = data['Herbivory'], poin_title = 'Global sample points')

box plot

In [ ]:
import seaborn as sns

a = data.loc[(data['class'] == 'Mixed')].index
data=data.drop(axis=0,index=a)
class_plot = pd.DataFrame({'herb': pd.Series(data.loc[(data['class'] == 'herb')]['Herbivory'].values),
                            'woody': pd.Series(data.loc[(data['class'] == 'woody')]['Herbivory'].values)})
fig3, ax3 = plt.subplots(1, 2, figsize=(14, 8))
# growf_plot.boxplot()
sns.boxplot(data = class_plot, ax = ax3[0]) # box
sns.violinplot(data = class_plot, ax=ax3[1]) # violin
ax3[1].set(ylim=(0, 100))
ax3[0].set(ylim=(0, 100))
ax3[0].set_xlabel("a")
ax3[0].set_ylabel("Herbivory")
ax3[1].set_xlabel("b")
ax3[1].set_ylabel("Herbivory")
plt.savefig(r'C:\\Users\\****\\photo\\herb_woody_box.png', dpi=600, bbox_inches = 'tight')


Kruskal-Wallis test by R

In [ ]:
# Define R path
import os
os.environ['R_HOME']= r'C:\\Program Files\\R\\R-4.2.2' # Change to your own version and address
os.environ['PATH'] = os.pathsep + r'C:\\Program Files\\R\\R-4.2.2\\bin\\X64\\' + ";" + os.environ["PATH"]
# Call R
from rpy2.robjects import r  # Calling R allows for writing some simple R code
from rpy2.robjects.packages import importr  # 'importr' like library in R
from rpy2.robjects import pandas2ri  # Convert Pandas objects to objects that can be read by R
from rpy2.robjects import globalenv
from rpy2.robjects.conversion import localconverter
import rpy2.robjects as ro
importr('stats')

data_test = data[['Herbivory', 'class']]
with localconverter(ro.default_converter + pandas2ri.converter):
    data_test = ro.conversion.py2rpy(data_test)

globalenv['data_test'] = data_test

rscript = """
kruskal.test(Herbivory~class, data=data_test)
"""
print(r(rscript))


## Data import and processing
### Read data
Read climate data with a resolution of 5 arc minutes, data source： WorldClim (https://www.worldclim.org/data/worldclim21.html)\
Using Geodetic Projection： EPSG:4326 (WGS84)

In [ ]:
import xarray as xr
import rioxarray as rixary  # rioxarray: reading Grid Files(tif) and Convert to Grid File(nc)

# Reading climate data
cli_colnames = ["AMT", "TDR", "Isoth", "Tseason", "TWM", "TCM", "ATR", "TWEQ", "TDQ", "TWQ", "TCQ", "AP", "PWM", "PDM",
            "Pseason", "PWeQ", "PDQ", "PWQ", "PCQ"]  # Climate variables
climate_x = [] 
for i in range(len(cli_colnames)):
    cli = rixary.open_rasterio(
        r'C:\\Users\\*******\\wc2.1_5m_bio\\wc2.1_5m_bio_' + str(i + 1) + r'.tif')
    climate_x.append(cli)
climate = xr.concat(climate_x, dim='band')
climate['band'] = cli_colnames


Obtain the biological community information of the global scale land Ecotope. source :World Wildlife Fund:https://www.worldwildlife.org/publications/terrestrial-ecoregions-of-the-world 

In [ ]:

biome_name = ["Tropical & Subtropical Moist Broadleaf Forests","Tropical & Subtropical Dry Broadleaf Forests",
              "Tropical & Subtropical Coniferous Forests","Temperate Broadleaf & Mixed Forests","Temperate Conifer Forests",
              "Boreal Forests/Taiga","Tropical & Subtropical Grasslands, Savannas & Shrublands","Temperate Grasslands, Savannas & Shrublands",
              "Flooded Grasslands & Savannas","Montane Grasslands & Shrublands","Tundra","Mediterranean Forests, Woodlands & Scrub",
              "Deserts & Xeric Shrublands","Mangroves"] # Biocommunity Types

bio_colnames = ['biome']
biome = rixary.open_rasterio(r'C:\\Users\\*****\\biome-new\\wwf_terr_ecos.tif') 
biome['band'] = bio_colnames

Read soil data with a resolution of 5 arc minutes, data source：Harmonized World Soil Database v1.2(https://www.fao.org/soils-portal/soil-survey/soil-maps-and-databases/harmonized-world-soil-database-v12/en/)/
Resampled by ArcGis, adjust the resolution to 5min(BILINEAR).

In [ ]:
# Read soil data
soil_colnames = ['T_GRAVEL','T_SAND','T_SILT','T_CLAY','T_REF_BULK_DENSITY','T_BULK_DENSITY','T_OC','T_PH_H2O','T_CEC_CLAY','T_CEC_SOIL',
            'T_BS','T_TEB','T_CACO3','T_CASO4','T_ESP','T_ECE']
soil_x=[] 
for soil_colname in soil_colnames:
    soi = rixary.open_rasterio(r'C:\\Users\\*****\\HWSD_5m\\data\\'+ soil_colname +r'_5m.tif')
    soil_x.append(soi)
soil = xr.concat(soil_x, dim='band')  
soil['band'] = soil_colnames


The land cover type. Data source：2020 MODIS MCD12Q1：https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.061/ \
Friedl, M., Sulla-Menashe, D. (2022). MODIS/Terra+Aqua Land Cover Type Yearly L3 Global 0.05Deg CMG V061 [Data set]. NASA EOSDIS Land Processes DAAC. Accessed 2023-03-28 from https://doi.org/10.5067/MODIS/MCD12C1.061/
Resampled by ArcGis, adjust the resolution to 5min(nearest).


In [ ]:
glc = rixary.open_rasterio(r'C:\\Users\\******\\MCD12Q1_v061_LC_Type\\MCD12Q1_v061_LC_5m_Type.tif')


### Add environmental data
Add soil data, climate data, and biological communities to the dataset based on latitude and longitude

In [ ]:
data_all=data

# Functions to extract index
def find_index(target, A):
    index_list=[]
    if len(A) < 1:
        print('The input is null.')
    else:
        for i in trange(len(target), desc='extract index'):
            index_list.append(A.index(min(A, key=lambda x: abs(x - target[i]))))
    return index_list

lon = climate['x'].values.tolist() # extract longitude
lat = climate['y'].values.tolist()

lon_indexs = find_index(data['Longitude'].tolist(), lon)
lat_indexs = find_index(data['Latitude'].tolist(), lat)


# Add climate data
for colname in cli_colnames:
    cli = []
    dat = climate.sel(band = colname).values
    for (lon_index,lat_index) in zip(lon_indexs,lat_indexs):
        cli.append(dat[lat_index, lon_index])
    data_all[colname] = cli

# Add soil data
for colname in soil_colnames:
    soi = []
    dat = soil.sel(band = colname).values
    for (lon_index,lat_index) in zip(lon_indexs,lat_indexs):
        soi.append(dat[lat_index, lon_index])
    data_all[colname] = soi



# Add biome
lon_bio = biome['x'].values.tolist()
lat_bio = biome['y'].values.tolist()

lon_bio_indexs = find_index(data['Longitude'].tolist(), lon_bio)
lat_bio_indexs = find_index(data['Latitude'].tolist(), lat_bio)

for colname in bio_colnames:
    bio = []
    dat = biome.sel(band = colname).values
    for (lon_bio_index,lat_bio_index) in zip(lon_bio_indexs,lat_bio_indexs):
        bio.append(dat[lat_bio_index, lon_bio_index])
    data_all[colname] = bio


# Add absolute latitude
data_all['abs_lat'] = abs(data[['Latitude']])
data_all['Lon'] = data[['Longitude']]
data_all

In [ ]:

# name for biome1-biome13
for i in range(1,14,1):
    exec("""biome{} = pd.Series(data.loc[(data_all['biome'] == i)]['Herbivory'].values)""".format(i))
# add biome1-biome13
biome_plot = pd.DataFrame({'1': biome1})
for i in range(2,14,1):
    biome_plot[f'{i}'] = eval(f'biome{i}')

fig4, ax4 = plt.subplots(2,1, figsize=(14,14))


sns.boxplot(data = biome_plot, ax = ax4[0])
sns.violinplot(data = biome_plot, width=1.5, ax=ax4[1])
ax4[1].set(ylim=(0, 100))
ax4[0].set_xlabel("a")
ax4[0].set_ylabel("Herbivory")
ax4[1].set_xlabel("b")
ax4[1].set_ylabel("Herbivory")
plt.savefig(r'C:\\Users\\*****\\photo\\biome_box.png', dpi=600, bbox_inches = 'tight')


Kruskal-Wallis test in different biome

In [ ]:
biome_index = (data_all['biome'] == 1)
for i in range(2,14,1):
    biome_index += (data_all['biome'] == i)
biome_test = data.loc[biome_index][['biome','Herbivory']]

with localconverter(ro.default_converter + pandas2ri.converter):
    biome_test = ro.conversion.py2rpy(biome_test)

globalenv['biome_test'] = biome_test
rscript = """
kruskal.test(Herbivory~biome, data=biome_test)
"""
print(r(rscript))


Kruskal-Wallis test among biomes

In [ ]:
rscript = """
p.test<-pairwise.wilcox.test(biome_test$Herbivory, reorder(biome_test$biome, -biome_test$Herbivory), p.adjust.method = "BH")
p.test$p.value
"""
print(r(rscript))

Removing useless values (refers to the loss of meaning in the values extracted from coordinates due to the accuracy of the data, such as extracting values from a point by the sea(usually very small, with an order of magnitude of$10^{-38}$))

In [ ]:
data_anals = data_all.drop(axis=0,index=(data_all.loc[((data_all['AMT'] < -1000)
                                                       + (data_all['T_GRAVEL'] < -1000)
                                                       + (data_all['T_CEC_CLAY'] < -1000))].index))
data_anals
data_anals.to_excel('C:/Users/*****/data_anals.xlsx')

All the sample points in analysis

In [ ]:
data_anals_herb = data_anals.loc[(data_anals['class'] == 'herb')]
data_anals_woody = data_anals.loc[(data_anals['class'] == 'woody')]

fig, ax = plt.subplots(figsize=(15, 12), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_xlim([-180, 180])
ax.set_ylim([-90, 90])
plt.title('Global sample points in training models')
ax.add_feature(cfeatur.OCEAN.with_scale('10m'), zorder=2)  
ax.add_feature(cfeatur.LAKES.with_scale('10m'), zorder=2)  
mapbar = ax.scatter(data_anals_herb['Longitude'], data_anals_herb['Latitude'], c='Gold', s=9,alpha=0.5)
ax.scatter(data_anals_woody['Longitude'], data_anals_woody['Latitude'], c='ForestGreen', s=9,alpha=0.5)
ax.gridlines(draw_labels=True, linestyle='--', lw = 0.3) 
herb_patch = ax.scatter([],[], c='Gold', marker = "o", s = 20, label = "herb")
woody_patch = ax.scatter([],[], c='ForestGreen', marker = "o",  s = 20, label = "woody")
plt.legend(handles=[herb_patch, woody_patch], loc="center left", fontsize=15,edgecolor='white', facecolor='white') 
plt.savefig(r'C:\\XXXXXX\\point_red_herb_woody1.png', dpi=600, bbox_inches = 'tight')


continent\
https://www.arcgis.com/home/item.html?id=3c4741e22e2e4af2bd4050511b9fc6ad#!

In [ ]:
continent = rixary.open_rasterio(r'C:XXXXX\\continent.tif')

lon_con = continent['x'].values.tolist() 
lat_con = continent['y'].values.tolist()
cont = continent.sel(band = 1).values
cont = np.array(cont, dtype=float)

lon_indexs_herb = find_index(data_anals_herb['Longitude'].tolist(), lon_con) 
lat_indexs_herb = find_index(data_anals_herb['Latitude'].tolist(), lat_con)

lon_indexs_woody = find_index(data_anals_woody['Longitude'].tolist(), lon_con) 
lat_indexs_woody = find_index(data_anals_woody['Latitude'].tolist(), lat_con)

# continent
cont_herb = []
for (lon_index,lat_index) in zip(lon_indexs_herb,lat_indexs_herb):
    cont_herb.append(cont[lat_index, lon_index])

cont_woody = []
for (lon_index,lat_index) in zip(lon_indexs_woody,lat_indexs_woody):
    cont_woody.append(cont[lat_index, lon_index])

belong_cont =dict({'herb':cont_herb,'woody':cont_woody})
cont_names = dict({'5':['South America'],'6':['Oceania'],'2':['North America'],'3':['Europe'],'1':['Asia'],'4':['Africa']})
# ({'4':['Africa'],'1':['Asia'],'3':['Europe'],'2':['North America'],'6':['Oceania'],'5':['South America']})
cont_count = pd.DataFrame()
for plant,belong in belong_cont.items():
    count_all=[]
    for key,value in cont_names.items():
        if value != ['Oceania']:
            count_i = belong.count(int(key))
            count_all.append(count_i)
        else:
            count_i = belong.count(int(key))+belong.count(7)
            count_all.append(count_i)
    cont_count[plant]=count_all
cont_count
nmae_x=['herbaceous','woody']
cont_names_list = ['South America','Oceania','North America','Europe','Asia','Africa']

from matplotlib import rcParams

config = {
    "font.family":'Times New Roman',  
    "axes.unicode_minus": False ,
    "font.size": 18
}
rgbcolor=['#5d73af', '#d38870', '#3c5488', '#267b6e', '#3e9bb1', '#cd422e']
# rcParams.update(config)
fig, ax = plt.subplots(figsize=(8,8))
p1 = ax.bar(nmae_x, cont_count.iloc[0,:], label=cont_names_list[0],color=rgbcolor[0])
p2 = ax.bar(nmae_x, cont_count.iloc[1,:], label=cont_names_list[1],color=rgbcolor[1],bottom=[sum(cont_count.iloc[:1,0].tolist()),sum(cont_count.iloc[:1,1].tolist())])
p3 = ax.bar(nmae_x, cont_count.iloc[2,:], label=cont_names_list[2],color=rgbcolor[2],bottom=[sum(cont_count.iloc[:2,0].tolist()),sum(cont_count.iloc[:2,1].tolist())])
p4 = ax.bar(nmae_x, cont_count.iloc[3,:], label=cont_names_list[3],color=rgbcolor[3],bottom=[sum(cont_count.iloc[:3,0].tolist()),sum(cont_count.iloc[:3,1].tolist())])
p5 = ax.bar(nmae_x, cont_count.iloc[4,:], label=cont_names_list[4],color=rgbcolor[4],bottom=[sum(cont_count.iloc[:4,0].tolist()),sum(cont_count.iloc[:4,1].tolist())])
p6 = ax.bar(nmae_x, cont_count.iloc[5,:], label=cont_names_list[5],color=rgbcolor[5],bottom=[sum(cont_count.iloc[:5,0].tolist()),sum(cont_count.iloc[:5,1].tolist())])
ax.set_ylabel('Site count')
ax.legend((p6[0], p5[0], p4[0],p3[0], p2[0],p1[0]), ('Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America'),frameon=False, prop=config)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.savefig(r'C:\\******\\continent.png', dpi=600, bbox_inches = 'tight')
plt.show()

biomeplot,AMT-AP

In [ ]:
data_biom = data_anals[['AMT', 'AP','class']]
with localconverter(ro.default_converter + pandas2ri.converter):
    data_biom = ro.conversion.py2rpy(data_biom)

globalenv['data_biom'] = data_biom

rscript = """library(plotbiomes)
library(readxl)
library(ggplot2)

whittaker_base_plot() +
  # add the temperature - precipitation data points
  geom_point(data = dat_biom, 
             aes(x = AMT, 
                 y = AP/10,color = class), 
             size   = 3,
             stroke = 2,
             alpha  = 0.5) +
  scale_color_manual(values = c("blue","red" ),name="")+ 
  theme_bw()+
  theme_classic()+ 
  theme(
    legend.justification = c(0, 1), # pick the upper left corner of the legend box and
    legend.position = c(0, 1), # adjust the position of the corner as relative to axis
    legend.background = element_rect(fill = NA), # transparent legend background
    legend.box = "horizontal", # horizontal arrangement of multiple legends
    legend.spacing.x = unit(0.5, units = "cm"), # horizontal spacing between legends
    legend.key.size = unit(0.8, "cm"),
    panel.grid = element_blank(), # eliminate grids
    text = element_text(family = 'serif',size=25)
  )

ggsave(filename = "C:/******/MAT_MAP.png",
       width = 8,
       height = 8,
       dpi = 600)"""

## Random Forest
### set parameter
ntrees for herb

In [ ]:
%%capture --no-display

# Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

location = ['abs_lat']
feature_name = cli_colnames + soil_colnames + location# name of all feature

cross = []
for i  in trange(0,200,10):
    rf = RandomForestRegressor(n_estimators=i+1, max_features= len(feature_name), n_jobs=-1, random_state=42)
    rf_fit = rf.fit(data_anals_herb[feature_name], data_anals_herb['Herbivory'])
    mse_score = ((rf_fit.predict(data_anals_herb[feature_name])-data_anals_herb['Herbivory'])**2).mean()
    cross.append(mse_score)
    
plt.plot(range(1,201,10),cross)
plt.xlabel('n_estimators')
plt.ylabel('MSE')
plt.title('a', loc = 'left', fontsize=30)
plt.title('The n-trees selection in random forest of herb')
plt.savefig(r'C:\\Users\\*****\\photo\\herb_ntrees.png', dpi=600, bbox_inches = 'tight')
plt.show()


ntrees for woody

In [ ]:
cross = []
for i  in trange(0,200,10):
    rf = RandomForestRegressor(n_estimators=i+1, max_features= len(feature_name), n_jobs=-1, random_state=42)
    rf_fit = rf.fit(data_anals_woody[feature_name], data_anals_woody['Herbivory'])
    mse_score = ((rf_fit.predict(data_anals_woody[feature_name])-data_anals_woody['Herbivory'])**2).mean()
    # cross_score = -cross_val_score(rf, data_anals[feature_name], data_anals['Herbivory'], cv=5, scoring='neg_mean_squared_error').mean() #交叉验证的MSE
    cross.append(mse_score)
    
plt.plot(range(1,201,10),cross)
plt.xlabel('n_estimators')
plt.ylabel('MSE')
plt.title('b', loc = 'left', fontsize=30)
plt.title('The n-trees selection in random forest of woody')
plt.savefig(r'C:\\Users\\loner\\lianxi\\photo\\woody_ntrees.png', dpi=600, bbox_inches = 'tight')
plt.show()

### Build model
importance for herb

In [ ]:
import matplotlib.patches as mpatches
from sklearn.ensemble import RandomForestRegressor


forest_herb = RandomForestRegressor(n_estimators=150, max_features=len(feature_name), random_state=42)
forest_herb.fit(data_anals_herb[feature_name], data_anals_herb['Herbivory'])

importance = forest_herb.feature_importances_
imp_herb = pd.DataFrame({'name': feature_name, 'importance': importance, 
    'color':['#778899']*len(cli_colnames) + ['#D2B48C']*len(soil_colnames)
            + ['#4682B4']*len(location) + ['#BC8F8F']*len(hum_colnames)}).sort_values(by=['importance'])  # Ascending order of importance


fig8, ax8 = plt.subplots(figsize=(6, 12))
bar = plt.barh(imp_herb['name'], imp_herb['importance'], color= imp_herb['color'],align="center", tick_label=imp_herb['name'])
cli_patch = mpatches.Patch(color='#778899', label='climate')
soi_patch = mpatches.Patch(color='#D2B48C', label='soil')
loc_patch = mpatches.Patch(color='#4682B4', label='location')
hum_patch = mpatches.Patch(color='#BC8F8F', label='human')
plt.legend(handles=[cli_patch, soi_patch, loc_patch, hum_patch], loc="lower right", borderaxespad=2)
plt.title('The importance of variables in herb')
plt.savefig(r'C:\\Users\\*****\\photo\\herb_imp.png', dpi=600, bbox_inches = 'tight')
plt.show()

importance for woody

In [ ]:
forest_woody = RandomForestRegressor(n_estimators=150, max_features=len(feature_name), random_state=42)
forest_woody.fit(data_anals_woody[feature_name], data_anals_woody['Herbivory'])

importance = forest_woody.feature_importances_
imp_woody = pd.DataFrame({'name': feature_name, 'importance': importance, 
    'color':['#778899']*len(cli_colnames) + ['#D2B48C']*len(soil_colnames)
            + ['#4682B4']*len(location) + ['#BC8F8F']*len(hum_colnames)}).sort_values(by=['importance'])  # 建立dataframe格式的数据，并将重要性升序排列  

fig9, ax9 = plt.subplots(figsize=(6, 12))
bar = plt.barh(imp_woody['name'], imp_woody['importance'], color= imp_woody['color'],align="center", tick_label=imp_woody['name'])
cli_patch = mpatches.Patch(color='#778899', label='climate') # 创建一个 artist用于添加图例
soi_patch = mpatches.Patch(color='#D2B48C', label='soil')
loc_patch = mpatches.Patch(color='#4682B4', label='location')
hum_patch = mpatches.Patch(color='#BC8F8F', label='human')
plt.legend(handles=[cli_patch, soi_patch, loc_patch, hum_patch], loc="lower right", borderaxespad=2)
plt.title('The importance of variables in woody')
plt.savefig(r'C:\\Users\\******\\photo\\woody_imp.png', dpi=600, bbox_inches = 'tight')
plt.show()


### Select Features
Select the most importance variables using the R-package VSURF\
#### select in herb

In [ ]:
importr('VSURF')

data_sel = (data_anals_herb[feature_name]).copy()
data_sel['Herbivory'] = data_anals_herb['Herbivory']
with localconverter(ro.default_converter + pandas2ri.converter):
    data_sel_r = ro.conversion.py2rpy(data_sel)


globalenv['data_sel_r'] = data_sel_r
rscript = """
set.seed(42)
rf_sel <- VSURF(Herbivory ~., data = data_sel_r) # , ncores= 4, parallel=TRUE
summary(rf_sel)
"""
r(rscript)


First，thresholding step

In [ ]:
rscript = """
figure = plot(rf_sel, step = "thres", imp.sd = FALSE, var.names = TRUE, cex.lab=1.4)
dev.off()
png(filename = 'C:/Users/loner/*****/pngvsurf/thres.png',width = 800,height = 500)
rf_prd <- colnames(data_sel_r)[rf_sel$varselect.thres] 
rf_prd
"""
r(rscript)

Second，interpretation step

In [ ]:
rscript = """
figure = plot(rf_sel, step = "interp", imp.sd = FALSE, var.names = TRUE, cex.lab=1.4)
dev.off()
png(filename = 'C:/Users/loner/*****/pngvsurf/interp.png',width = 800,height = 500)
rf_inter <- colnames(data_sel_r)[rf_sel$varselect_interp]
rf_inter
"""
r(rscript)

Third，prediction step

In [ ]:
rscript = """
figure = plot(rf_sel, step = "pred", imp.sd = FALSE, var.names = TRUE, cex.lab=1.4)
title(main="a", cex.main=3, adj=0)
dev.off()
png(filename = 'C:/Users/loner/*******/pngvsurf/pred.png',width = 800,height = 500)
rf_prd <- colnames(data_sel_r)[rf_sel$varselect.pred]
rf_prd
"""
print(r(rscript))

#### select in woody

In [ ]:
importr('VSURF')

data_sel = (data_anals_woody[feature_name]).copy()
data_sel['Herbivory'] = data_anals_woody['Herbivory']
with localconverter(ro.default_converter + pandas2ri.converter):
    data_sel_r = ro.conversion.py2rpy(data_sel)


globalenv['data_sel_r'] = data_sel_r
rscript = """
set.seed(42)
rf.sel <- VSURF(Herbivory ~., data = data_sel_r)
summary(rf.sel)
"""
r(rscript)


First，thresholding step

In [ ]:
rscript = """
figure = plot(rf.sel, step = "thres", imp.sd = FALSE, var.names = TRUE, cex.lab=1.4)
dev.off()
png(filename = 'C:/Users/loner/*******/pngvsurf/thres.png',width = 800,height = 500)
rf.prd <- colnames(data_sel_r)[rf.sel$varselect.thres]
rf.prd
"""
r(rscript)

Second，interpretation step

In [ ]:
rscript = """
figure = plot(rf.sel, step = "interp", imp.sd = FALSE, var.names = TRUE, cex.lab=1.4)
dev.off()
png(filename = 'C:/Users/loner/********/pngvsurf/interp.png',width = 800,height = 500)
rf.prd <- colnames(data_sel_r)[rf.sel$varselect.interp]
rf.prd
"""
r(rscript)

Third，prediction step

In [ ]:
rscript = """
figure = plot(rf.sel, step = "pred", imp.sd = FALSE, var.names = TRUE, cex.lab=1.4)
title(main="b", cex.main=3, adj=0)
dev.off()
png(filename = 'C:/Users/******/pngvsurf/pred.png',width = 800,height = 500)
rf.prd <- colnames(data_sel_r)[rf.sel$varselect.pred]
rf.prd
"""
print(r(rscript))

##  Relationships between herbivory and the most important predictor variables
###  For herb plants
Linear models

In [ ]:
# "AP"         "T_CEC_CLAY" "PDQ"        "TDR"        "TWEQ"      
# "abs_lat" 

plt.rcParams['axes.unicode_minus'] = False # Display negative sign

selec_fec_herb = ["AP", "T_CEC_CLAY", "PDQ", "TDR", "TWEQ", "abs_lat"]
selec_fec_herb_unit = ['mm', 'cmol/kg', 'mm','°C','°C','°']
sort = ['a','b','c','d','e','f','g','h']
selec_herb_xname = []
selec_herb_ximp = []
# importance of variables
for name in selec_fec_herb:
    selec_herb_xname.append(imp_herb.loc[imp_herb['name'] ==name]['name'].values.tolist()[0])
    selec_herb_ximp.append(round(imp_herb.loc[imp_herb['name'] ==name]['importance'].values.tolist()[0],3))
    
selec_herb_x = pd.DataFrame({'name': selec_herb_xname, 'unit': selec_fec_herb_unit,
                             'importance': selec_herb_ximp}).sort_values(by=['importance'], ascending = False)
selec_herb_x.index = range(len(selec_herb_x))
# linear models
def scatter_plot(xlabel, ylabel, xvalue, yvalue, data):
    sns.regplot(x=xvalue, y=yvalue, data=data[[xvalue, yvalue]],line_kws={'linewidth':2,'color':'red'}, scatter_kws={'s':20}) 
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel(ylabel, fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)


ylable_on = [1,4,7]

for i in range(len(selec_fec_herb)):
    # plt.subplot(2,3,i+1)
    plt.figure(figsize=(6,6))
    plt.tight_layout()
    if i+1 in ylable_on:
        if selec_herb_x['unit'][i] == '':
            scatter_plot(xlabel = selec_herb_x['name'][i], ylabel = 'Herbivory', xvalue = selec_herb_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_herb)
            
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\*******\\photo\\herb'+selec_herb_x['name'][i]+'linear.png', dpi=600, bbox_inches='tight')
        else:
            scatter_plot(xlabel = selec_herb_x['name'][i] + '('+selec_herb_x['unit'][i]+')', ylabel = 'Herbivory', xvalue = selec_herb_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_herb)
            
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\*******\\photo\\herb'+selec_herb_x['name'][i]+'linear.png', dpi=600, bbox_inches='tight')
    else:
        if selec_herb_x['unit'][i] == '':
            scatter_plot(xlabel = selec_herb_x['name'][i], ylabel = '', xvalue = selec_herb_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_herb)
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\*******\\photo\\herb'+selec_herb_x['name'][i]+'linear.png', dpi=600, bbox_inches='tight')
        else:
            scatter_plot(xlabel = selec_herb_x['name'][i] + '('+selec_herb_x['unit'][i]+')', ylabel = '', xvalue = selec_herb_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_herb)
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\*******\\photo\\herb'+selec_herb_x['name'][i]+'linear.png', dpi=600, bbox_inches='tight')
        

 non-linear models

In [ ]:
plt.rcParams['axes.unicode_minus'] = False # Display negative sign

selec_fec_herb = ["AP", "T_CEC_CLAY", "PDQ", "TDR", "TWEQ", "abs_lat"]
selec_fec_herb_unit = ['mm', 'cmol/kg', 'mm','°C','°C','°']
selec_herb_xname = []
selec_herb_ximp = []
# importance of variables
for name in selec_fec_herb:
    selec_herb_xname.append(imp_herb.loc[imp_herb['name'] ==name]['name'].values.tolist()[0])
    selec_herb_ximp.append(round(imp_herb.loc[imp_herb['name'] ==name]['importance'].values.tolist()[0],3))
    
selec_herb_x = pd.DataFrame({'name': selec_herb_xname, 'unit': selec_fec_herb_unit,
                             'importance': selec_herb_ximp}).sort_values(by=['importance'], ascending = False)
selec_herb_x.index = range(len(selec_herb_x)) 

# highest degree of 2
def scatter_plot(xlabel, ylabel, xvalue, yvalue, data):
    # order=2 means quadratic fitting
    sns.lmplot(x=xvalue, y=yvalue, data=data[[xvalue, yvalue]],order=2,line_kws={'linewidth':2,'color':'red'}, scatter_kws={'s':20})
    sns.despine(top=False, right=False)
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel(ylabel, fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)


ylable_on = [1,4,7] 

for i in range(len(selec_fec_herb)):
    plt.figure(figsize=(6,6))
    plt.tight_layout()
    if i+1 in ylable_on:
        if selec_herb_x['unit'][i] == '':
            
            scatter_plot(xlabel = selec_herb_x['name'][i], ylabel = 'Herbivory', xvalue = selec_herb_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_herb)
            
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\******\\photo\\herb'+selec_herb_x['name'][i]+'nonlinear.png', dpi=600)
        else:
            
            scatter_plot(xlabel = selec_herb_x['name'][i] + '('+selec_herb_x['unit'][i]+')', ylabel = 'Herbivory', xvalue = selec_herb_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_herb)
            
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\******\\photo\\herb'+selec_herb_x['name'][i]+'nonlinear.png', dpi=600)
    else:
        if selec_herb_x['unit'][i] == '':
            
            scatter_plot(xlabel = selec_herb_x['name'][i], ylabel = '', xvalue = selec_herb_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_herb)
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\******\\photo\\herb'+selec_herb_x['name'][i]+'nonlinear.png', dpi=600)
        else:
            
            scatter_plot(xlabel = selec_herb_x['name'][i] + '('+selec_herb_x['unit'][i]+')', ylabel = '', xvalue = selec_herb_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_herb)
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\******\\photo\\herb'+selec_herb_x['name'][i]+'nonlinear.png', dpi=600)


AIC and $R^2$

In [ ]:
x1 = []
x2 = []
x3 = []
x4 = []
for i in range(len(selec_fec_herb)):
    data_sel_aic = pd.DataFrame({'name_aic': (data_anals_herb[selec_herb_x['name'][i]]).copy()})
    data_sel_aic['Herbivory'] = data_anals_herb['Herbivory']
    # data_sel_aic.columns=[['name_aic','Herbivory' ]]

    with localconverter(ro.default_converter + pandas2ri.converter):
        data_sel_aic_r = ro.conversion.py2rpy(data_sel_aic)

    globalenv['data_sel_aic_r'] = data_sel_aic_r

    rscript = """
    reg1<- lm(Herbivory~name_aic,data = data_sel_aic_r)
    reg2<- lm(Herbivory~poly(name_aic,2,raw = TRUE),data = data_sel_aic_r)
    x1<- AIC(reg1)
    x2<- AIC(reg2)
    x3<- summary(reg1)$r.squared
    x4<- summary(reg2)$r.squared
    """
    r(rscript)
    x1.append(r['x1'][0])
    x2.append(r['x2'][0])
    x3.append(r['x3'][0])
    x4.append(r['x4'][0])

herb_aic = pd.DataFrame({'name': selec_herb_x['name'], 'x1':x1, 'x2':x2, 'x3':x3, 'x4':x4})
herb_aic.to_excel('C:/Users/*****/photo/data_anals_herb_result.xlsx')
herb_aic

###  For woody plants
Linear models

In [ ]:
# "PDM"     "TWM"     "PWM"     "PDQ"     "TCM"     "Isoth"   "TDR"    
# [8] "abs_lat"
selec_fec_woody = ["PDM", "TWM", "PWM", "PDQ", "TCM", "Isoth", "TDR", "abs_lat"]
selec_fec_woody_unit = ['mm', '°C', 'mm', 'mm', '°C', '', '°C', '°']
selec_woody_xname = []
selec_woody_ximp = []
# importance of variables
for name in selec_fec_woody:
    selec_woody_xname.append(imp_woody.loc[imp_woody['name'] ==name]['name'].values.tolist()[0])
    selec_woody_ximp.append(round(imp_woody.loc[imp_woody['name'] ==name]['importance'].values.tolist()[0], 3))
    
selec_woody_x = pd.DataFrame({'name': selec_woody_xname, 'unit': selec_fec_woody_unit,
                             'importance': selec_woody_ximp}).sort_values(by=['importance'],ascending = False)
selec_woody_x.index = range(len(selec_woody_x)) 
# Linear models
def scatter_plot(xlabel, ylabel, xvalue, yvalue, data):
    sns.regplot(x=xvalue, y=yvalue, data=data[[xvalue, yvalue]],line_kws={'linewidth':2,'color':'red'}, scatter_kws={'s':20}) 
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel(ylabel, fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

for i in range(len(selec_fec_woody)):
    plt.figure(figsize=(6,6))
    plt.tight_layout()
    if i+1 in ylable_on:
        if selec_woody_x['unit'][i] == '':
            scatter_plot(xlabel = selec_woody_x['name'][i], ylabel = 'Herbivory', xvalue = selec_woody_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_woody)
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\******\\photo\\woody'+selec_woody_x['name'][i]+'linear.png', dpi=600,bbox_inches = 'tight')
        else:
            scatter_plot(xlabel = selec_woody_x['name'][i] + '('+selec_woody_x['unit'][i]+')', ylabel = 'Herbivory', xvalue = selec_woody_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_woody)
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\******\\photo\\woody'+selec_woody_x['name'][i]+'linear.png', dpi=600,bbox_inches = 'tight')
    else:
        if selec_woody_x['unit'][i] == '':
            scatter_plot(xlabel = selec_woody_x['name'][i], ylabel = '', xvalue = selec_woody_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_woody)
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\******\\photo\\woody'+selec_woody_x['name'][i]+'linear.png', dpi=600,bbox_inches = 'tight')
        else:
            scatter_plot(xlabel = selec_woody_x['name'][i] + '('+selec_woody_x['unit'][i]+')', ylabel = '', xvalue = selec_woody_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_woody)
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\******\\photo\\woody'+selec_woody_x['name'][i]+'linear.png', dpi=600,bbox_inches = 'tight')
            

non-linear models

In [ ]:
# highest degree of 2
def scatter_plot(xlabel, ylabel, xvalue, yvalue, data):
    # order=2 means quadratic fitting
    sns.lmplot(x=xvalue, y=yvalue, data=data[[xvalue, yvalue]],order=2,line_kws={'linewidth':2,'color':'red'}, scatter_kws={'s':20}) # , fit_reg = False该参数不显示回归线
    sns.despine(top=False, right=False)
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel(ylabel, fontsize=15)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)


ylable_on = [1,4,7]

for i in range(len(selec_fec_woody)):
    plt.figure(figsize=(6,6))
    plt.tight_layout()
    if i+1 in ylable_on:
        if selec_woody_x['unit'][i] == '':
            
            scatter_plot(xlabel = selec_woody_x['name'][i], ylabel = 'Herbivory', xvalue = selec_woody_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_woody)
            
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\*****\\photo\\woody'+selec_woody_x['name'][i]+'nonlinear.png', dpi=600)
        else:
            
            scatter_plot(xlabel = selec_woody_x['name'][i] + '('+selec_woody_x['unit'][i]+')', ylabel = 'Herbivory', xvalue = selec_woody_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_woody)
            
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\*****\\photo\\woody'+selec_woody_x['name'][i]+'nonlinear.png', dpi=600)
    else:
        if selec_woody_x['unit'][i] == '':
            
            scatter_plot(xlabel = selec_woody_x['name'][i], ylabel = '', xvalue = selec_woody_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_woody)
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\*****\\photo\\woody'+selec_woody_x['name'][i]+'nonlinear.png', dpi=600)
        else:
            
            scatter_plot(xlabel = selec_woody_x['name'][i] + '('+selec_woody_x['unit'][i]+')', ylabel = '', xvalue = selec_woody_x['name'][i], yvalue = 'Herbivory',
                        data = data_anals_woody)
            plt.title(sort[i], x= -0.12, y=0.95, fontdict={'size':25, 'weight':'bold'})
            plt.ylim(0,100)
            plt.savefig(r'C:\\Users\\*****\\photo\\woody'+selec_woody_x['name'][i]+'nonlinear.png', dpi=600)

AIC and $R^2$

In [ ]:
x1 = []
x2 = []
x3 = []
x4 = []
for i in range(len(selec_fec_woody)):
    data_sel_aic = pd.DataFrame({'name_aic': (data_anals_woody[selec_woody_x['name'][i]]).copy()})
    data_sel_aic['Herbivory'] = data_anals_woody['Herbivory']
    # data_sel_aic.columns=[['name_aic','Herbivory' ]]

    with localconverter(ro.default_converter + pandas2ri.converter):
        data_sel_aic_r = ro.conversion.py2rpy(data_sel_aic)

    globalenv['data_sel_aic_r'] = data_sel_aic_r

    rscript = """
    reg1<- lm(Herbivory~name_aic,data = data_sel_aic_r)
    reg2<- lm(Herbivory~poly(name_aic,2,raw = TRUE),data = data_sel_aic_r)
    x1<- AIC(reg1)
    x2<- AIC(reg2)
    x3<- summary(reg1)$r.squared
    x4<- summary(reg2)$r.squared
    """
    r(rscript)
    x1.append(r['x1'][0])
    x2.append(r['x2'][0])
    x3.append(r['x3'][0])
    x4.append(r['x4'][0])

woody_aic = pd.DataFrame({'name': selec_woody_x['name'], 'x1':x1, 'x2':x2, 'x3':x3, 'x4':x4})
woody_aic.to_excel('C:/Users/loner/lianxi/photo/data_anals_woody_result.xlsx')
woody_aic

## Global prediction

### Data preparation

In [ ]:
lon, lat=np.meshgrid(climate['x'], climate['y']) # Generate grid with latitude and longitude

def list_nonest(lis):
    new_list = []
    for i in range(len(lis)):
        new_list += lis[i]
    return new_list

# climate data
data_envri = pd.DataFrame()
for colname in cli_colnames:
    dat = climate.loc[colname].values
    data_envri[colname] = list_nonest(dat.tolist())

# soil data
for colname in soil_colnames:
    dat = soil.loc[colname].values
    data_envri[colname] = list_nonest(dat.tolist())

# Absolute latitude
abs_lat0 = abs(lat).tolist()
data_envri['abs_lat'] = list_nonest(abs_lat0)

# longitude
Lon = lon.tolist()
data_envri['Lon'] = list_nonest(Lon)

data_envri = pd.read_csv('C:/Users/*****/data_envri.csv')

###  Global maps of herbivory
Draw a prediction map based on the distribution of herb or woody plants


In [ ]:
# NA means those areas where (1) data in any of the required predictors were missing
#                            (2) land cover type was different to the plant type we predict
# function to assign these areas the value NA
def replac_to_na(herb_woody, predict):
    mask = (np.isnan(herb_woody))
    predict[mask] = np.nan
    # climate
    for colname in cli_colnames:
        dat = climate.loc[colname].values
        mask = (dat < -10000)
        predict[mask] = np.nan
        
    # soil
    for colname in soil_colnames:
        dat = soil.loc[colname].values
        mask = (dat < -10000)
        predict[mask] = np.nan
    # human foot
    humfoot_na = np.array(list_nest(data_envri["human_foot"], m, n))
    mask = (humfoot_na < -10000)
    predict[mask] = np.nan
    return predict

glc_plant = glc.sel(band = 1).values

# region of herb is 8-11
glc_herb = np.copy(glc_plant)
glc_herb = np.array(glc_herb, dtype=float)
glc_herb[glc_herb < 8] = np.nan
glc_herb[glc_herb > 11] = np.nan

# region of woody is 1-7
glc_woody = np.copy(glc_plant)
glc_woody = np.array(glc_woody, dtype=float)
glc_woody[glc_woody > 7] = np.nan

### Prediction model of Random forest
#### Predictive model for herb plants


In [ ]:
from sklearn.ensemble import RandomForestRegressor
# fit
forest_pre_herb = RandomForestRegressor(n_estimators=150, max_features=len(selec_fec_herb), random_state=42, n_jobs=-1)
forest_pre_herb.fit(data_anals_herb[selec_fec_herb], data_anals_herb['Herbivory'])
pl_fit0_herb = forest_pre_herb.predict(data_envri[selec_fec_herb])

# Structural transformation of data
def list_nest(lis, m, n):
    lis = lis.tolist()
    lis_nest = []
    for i in range(n, (m + 1) * n, n):
        lis_nest.append(lis[i - n:i])
    return lis_nest

# shape
m, n = len(climate['y']), len(climate['x'])
# Convert to array
pl_fit_herb = replac_to_na(glc_herb, np.array(list_nest(pl_fit0_herb, m, n)))
pl_fit_herb

#### Plot map

In [ ]:
import cartopy.crs as ccrs 
import cartopy.feature as cfeatur
import cartopy.io as cio 

config = {
    "font.family":'Times New Roman',
    "mathtext.fontset": 'stix',
    "font.serif": ['SimSun'],
}
mpl.rcParams.update(config)
mpl.rcParams['axes.unicode_minus']=False

# function to plot map
def pre_plot(pre_value:np.array, colormap, col_min, col_max, pre_title:str):
    fig, ax = plt.subplots(figsize=(15, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    plt.title(pre_title)
    mapbar1 = ax.imshow(pre_value, cmap=colormap, extent=[-180,180,-90,90], vmin=col_min, vmax=col_max) 
    
    ax.add_feature(cfeatur.OCEAN.with_scale('10m'), zorder=2)
    ax.add_feature(cfeatur.LAKES.with_scale('10m'), zorder=2)
    ax.gridlines(draw_labels=True, linestyle='--', lw=0.3) 
    cax = fig.add_axes([ax.get_position().x0, ax.get_position().y0-0.05, ax.get_position().width, 0.02]) 
    clb = plt.colorbar(mapbar1, cax=cax, orientation = 'horizontal')
    clb.set_label('Herbivory')
    plt.savefig(r'C:\\Users\\****\\photo\\' + pre_title + '.png', dpi=1200, bbox_inches = 'tight')

pre_plot(pre_value = pl_fit_herb, colormap = newcmap, col_min = 0, col_max = 20, pre_title = 'Global herbivory prediction of herb')


#### Predictive model for woody plants

In [ ]:
# fit
forest_pre_woody = RandomForestRegressor(n_estimators=150, max_features=len(selec_fec_woody), random_state=42, n_jobs=-1)
forest_pre_woody.fit(data_anals_woody[selec_fec_woody], data_anals_woody['Herbivory'])
pl_fit0_woody = forest_pre_woody.predict(data_envri[selec_fec_woody])

# Structural transformation of data
def list_nest(lis, m, n):
    lis = lis.tolist()
    lis_nest = []
    for i in range(n, (m + 1) * n, n):
        lis_nest.append(lis[i - n:i])
    return lis_nest

# shape
m, n = len(climate['y']), len(climate['x'])
# Convert to array
pl_fit_woody = replac_to_na(glc_woody, np.array(list_nest(pl_fit0_woody, m, n)))
pl_fit_woody

plot map

In [ ]:
pre_plot(pre_value = pl_fit_woody, colormap = newcmap, col_min = 0, col_max = 80, pre_title = 'Global herbivory prediction of woody')

prediction of herb and woody plants in one map

In [ ]:
fig13, ax13 = plt.subplots(figsize=(15, 12), subplot_kw={'projection': ccrs.PlateCarree()})
plt.title('Global herbivory prediction')
mapbar1 = ax13.imshow(pl_fit_herb, cmap=newcmap, extent=[-180,180,-90,90], vmin=0, vmax=80)  
ax13.imshow(pl_fit_woody, cmap=newcmap, extent=[-180,180,-90,90], vmin=0, vmax=80)  
ax13.add_feature(cfeatur.OCEAN.with_scale('10m'), zorder=2) 
ax13.add_feature(cfeatur.LAKES.with_scale('10m'), zorder=2)  

ax13.gridlines(draw_labels=True, linestyle='--', lw = 0.3)  
cax = fig13.add_axes([ax13.get_position().x0, ax13.get_position().y0-0.05, ax13.get_position().width, 0.02])  
clb = plt.colorbar(mapbar1, cax=cax, orientation = 'horizontal')
clb.set_label('Herbivory')


plt.savefig(r'C:\\Users\\*****\\photo\\global prediction.png', dpi=1200, bbox_inches = 'tight')


Comparison between the full model and the final model

In [ ]:
# Mean of squared residuals
def mssr(fit,true):
    return ((fit-true)**2).mean()

mse_score_full_herb = mssr(fit = forest_herb.predict(data_anals_herb[feature_name]), true = data_anals_herb['Herbivory'])
mse_score_final_herb = mssr(fit = forest_pre_herb.predict(data_anals_herb[selec_fec_herb]), true = data_anals_herb['Herbivory'])

mse_score_full_woody = mssr(fit = forest_woody.predict(data_anals_woody[feature_name]), true = data_anals_woody['Herbivory'])
mse_score_final_woody = mssr(fit = forest_pre_woody.predict(data_anals_woody[selec_fec_woody]), true = data_anals_woody['Herbivory'])

var_ex_full_herb = 1-mse_score_full_herb/mssr(fit = data_anals_herb['Herbivory'], true = (data_anals_herb['Herbivory']).mean())
var_ex_final_herb = 1-mse_score_final_herb/mssr(fit = data_anals_herb['Herbivory'], true = (data_anals_herb['Herbivory']).mean())

var_ex_full_woody = 1-mse_score_full_woody/mssr(fit = data_anals_woody['Herbivory'], true = (data_anals_woody['Herbivory']).mean())
var_ex_final_woody = 1-mse_score_final_woody/mssr(fit = data_anals_woody['Herbivory'], true = (data_anals_woody['Herbivory']).mean())
    
print('MSE of full model for herb:', mse_score_full_herb, 'MSE of final model for herb:', mse_score_final_herb)
print('MSE of full model for woody:', mse_score_full_woody, 'MSE of final model for woody:', mse_score_final_woody)

print('R^2 of full model for herb:', var_ex_full_herb, 'R^2 of final model for herb:', var_ex_final_herb)
print('R^2 of full model for woody:', var_ex_full_woody, 'R^2 of final model for woody:', var_ex_final_woody)


100 K-fold cross validations

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cross = []
for i  in trange(0,100,1):
    kf = KFold(n_splits=10, random_state=i+1,shuffle=True)
    cross_score = -cross_val_score(forest_pre_herb, data_anals_herb[selec_fec_herb], data_anals_herb['Herbivory'], cv=kf, scoring='neg_mean_squared_error').mean()
    cross.append(cross_score)
    
plt.plot(range(1,101,1),cross)
plt.xlabel('n')
plt.ylabel('MSE')
plt.title('a', loc = 'left', fontsize=30)
plt.title('10-fold cross validation for herb')
plt.savefig(r'C:\\Users\\*****\\photo\\cross validations_herb.png', dpi=600, bbox_inches = 'tight')
plt.show()




In [ ]:
cross = []
for i  in trange(0,100,1):
    kf = KFold(n_splits=10, random_state=i+1,shuffle=True)
    cross_score = -cross_val_score(forest_pre_woody, data_anals_woody[selec_fec_woody], data_anals_woody['Herbivory'], cv=kf, scoring='neg_mean_squared_error').mean()
    cross.append(cross_score)
    
plt.plot(range(1,101,1),cross)
plt.xlabel('n')
plt.ylabel('MSE')
plt.title('b', loc = 'left', fontsize=30)
plt.title('10-fold cross validation for woody')
plt.savefig(r'C:\\Users\\*****\\photo\\cross validations_woody.png', dpi=600, bbox_inches = 'tight')
plt.show()

### Global prediction in the context of future climate change
#### Sustainable development(ssp1.26)
Read global climate prediction data under the ACCESS-CM2 model, resolution 5 arc minutes, data source： WorldClim (https://worldclim.org/data/cmip6/cmip6_clim5m.html)


In [ ]:
from sklearn.impute import SimpleImputer 

clim_ssp126 = rixary.open_rasterio(r'C:\\Users\\*****\\wc2.1_5m_bio_ssp126_2021-2040\\wc2.1_5m_bioc_ACCESS-CM2_ssp126_2021-2040.tif')
clim_ssp126['band'] = cli_colnames

def list_nonest(lis):
    new_list = []
    for i in range(len(lis)):
        new_list += lis[i]
    return new_list

# Replacing climate data with future data
data_envri_ssp126 = data_envri.copy()
for colname in cli_colnames:
    dat = clim_ssp126.loc[colname].values
    imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-3.4e+38)
    dat = imp.fit_transform(dat)
    data_envri_ssp126[colname] = list_nonest(dat.tolist())


Prediction of herb

In [ ]:
pl_fit_ssp1260_herb = forest_pre_herb.predict(data_envri_ssp126[selec_fec_herb])
pl_fit_ssp126_herb = replac_to_na(glc_herb, np.array(list_nest(pl_fit_ssp1260_herb, m, n)))

pre_plot(pre_value = pl_fit_ssp126_herb, colormap = newcmap, col_min = 0, col_max = 20, pre_title = 'Global herbivory prediction of herb under ssp1.26')


Herbivory change of herb 

In [ ]:
# Herbivory change
pl_fit0_herb_rate = pl_fit_ssp1260_herb - pl_fit0_herb
pl_herb_rate126 = replac_to_na(glc_herb, np.array(list_nest(pl_fit0_herb_rate, m, n)))

pre_plot(pre_value = pl_herb_rate126, colormap = 'seismic', col_min = -10, col_max = 10, pre_title = 'Global herbivory change of herb under ssp1.26')

Prediction of woody

In [ ]:
pl_fit_ssp1260_woody = forest_pre_woody.predict(data_envri_ssp126[selec_fec_woody])
pl_fit_ssp126_woody = replac_to_na(glc_woody, np.array(list_nest(pl_fit_ssp1260_woody, m, n)))

pre_plot(pre_value = pl_fit_ssp126_woody, colormap = newcmap, col_min = 0, col_max = 80, pre_title = 'Global herbivory prediction of woody under ssp1.26')

Herbivory change of woody

In [ ]:
pl_fit0_woody_rate = pl_fit_ssp1260_woody - pl_fit0_woody
pl_woody_rate126 = replac_to_na(glc_woody, np.array(list_nest(pl_fit0_woody_rate, m, n)))

pre_plot(pre_value = pl_woody_rate126, colormap = 'seismic', col_min = -20, col_max = 20, pre_title = 'Global herbivory change of woody under ssp1.26')

#### Traditional fossil energy(ssp5.85)
Read global climate prediction data under the ACCESS-CM2 model, resolution 5 arc minutes, data source： WorldClim (https://worldclim.org/data/cmip6/cmip6_clim5m.html)

In [ ]:

clim_ssp585 = rixary.open_rasterio(r'C:\\Users\\******\\wc2.1_5m_bio_ssp585_2021-2040\\wc2.1_5m_bioc_ACCESS-CM2_ssp585_2021-2040.tif') 
clim_ssp585['band'] = cli_colnames

def list_nonest(lis):
    new_list = []
    for i in range(len(lis)):
        new_list += lis[i]
    return new_list

# Replacing climate data with future data
data_envri_ssp585 = data_envri.copy()
for colname in cli_colnames:
    dat = clim_ssp585.loc[colname].values
    imp = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-3.4e+38)
    dat = imp.fit_transform(dat)
    data_envri_ssp585[colname] = list_nonest(dat.tolist())

Prediction of herb

In [ ]:
pl_fit_ssp5850_herb = forest_pre_herb.predict(data_envri_ssp585[selec_fec_herb])
pl_fit_ssp585_herb = replac_to_na(glc_herb, np.array(list_nest(pl_fit_ssp5850_herb, m, n)))

pre_plot(pre_value = pl_fit_ssp585_herb, colormap = newcmap, col_min = 0, col_max = 20, pre_title = 'Global herbivory prediction of herb under ssp5.85')

Herbivory change of herb 

In [ ]:
pl_fit0_herb_rate = pl_fit_ssp5850_herb - pl_fit0_herb
pl_herb_rate585 = replac_to_na(glc_herb, np.array(list_nest(pl_fit0_herb_rate, m, n)))

pre_plot(pre_value = pl_herb_rate585, colormap = 'seismic', col_min = -10, col_max = 10, pre_title = 'Global herbivory change of herb under ssp5.85')

Prediction of woody

In [ ]:
pl_fit_ssp5850_woody =forest_pre_woody.predict(data_envri_ssp585[selec_fec_woody])
pl_fit_ssp585_woody = replac_to_na(glc_woody, np.array(list_nest(pl_fit_ssp5850_woody, m, n)))

pre_plot(pre_value = pl_fit_ssp585_woody, colormap = newcmap, col_min = 0, col_max = 80, pre_title = 'Global herbivory prediction of woody under ssp5.85')

Herbivory change of woody

In [ ]:
pl_fit0_woody_rate = pl_fit_ssp5850_woody - pl_fit0_woody
pl_woody_rate585 = replac_to_na(glc_woody, np.array(list_nest(pl_fit0_woody_rate, m, n)))

pre_plot(pre_value = pl_woody_rate585, colormap = 'seismic', col_min = -20, col_max = 20, pre_title = 'Global herbivory change of woody under ssp5.85')

#### Comparison between Sustainable Development and Fossil Fuel 
Herb

In [ ]:
pl_fit0_herb_126_585 = pl_fit_ssp1260_herb - pl_fit_ssp5850_herb
pl_herb_rate = replac_to_na(glc_herb, np.array(list_nest(pl_fit0_herb_126_585, m, n)))

pre_plot(pre_value = pl_herb_rate, colormap = 'seismic', col_min = -10, col_max = 10, pre_title = 'Global herbivory change of herb between ssp1.26 and ssp5.85')

Woody

In [ ]:
pl_fit0_woody_126_585 = pl_fit_ssp1260_woody - pl_fit_ssp5850_woody
pl_woody_rate = replac_to_na(glc_woody, np.array(list_nest(pl_fit0_woody_126_585, m, n)))

pre_plot(pre_value = pl_woody_rate, colormap = 'seismic', col_min = -10, col_max = 10, pre_title = 'Global herbivory change of woody between ssp1.26 and ssp5.85')